In [2]:
import boto3
import pandas as pd
from io import BytesIO
from datetime import datetime

endpoint_url =  'http://localhost:9000'
aws_access_key_id = 'teste'
aws_secret_access_key = 'OH9PTqKV1FRWV5fmWAMAEfUh05JWR1TOESxIhMww'
bucket_name = 'tiradentes'
filename = 'detailed_expens/tiradentes-detailed_expens-2023-12-06'

s3_resource = boto3.resource(
    's3', 
    endpoint_url=endpoint_url,
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key,
    aws_session_token=None,
)

bucket = s3_resource.Bucket(bucket_name)
    
obj = s3_resource.Object(bucket_name, filename)


In [34]:
df = pd.read_parquet(BytesIO(obj.get()['Body'].read()))
df.head()

CITY_EXPENSE_COLUMNS_RENAME = {
    "Número": "number",
    "Exercício": "year",
    "Tipo": "type",
    "Data do Empenho": "effort_date",
    "Liquidação": "settlement_date",
    "Pagamento": "payment_date",
    "Unidade": "unit",
    "Função": "function",
    "Subfunção": "subfunction",
    "Programa": "program",
    "Projeto Atividade": "activity",
    "Categoria Econômica": "economic_category",
    "Fonte de Recurso": "resource_source",
    "CO TCE": "co_tce",
    "CO AUX": "co_aux",
    "Valor": "value",
    "Beneficiário": "creditor",
    "CPF/CNPJ": "cpf_cnpj",
    "Histórico": "history",
}

df.rename(columns=CITY_EXPENSE_COLUMNS_RENAME, inplace=True)
df['effort_date'] = pd.to_datetime(df['effort_date'], format="%d/%m/%Y")
df['settlement_date'] = pd.to_datetime(df['settlement_date'], format="%d/%m/%Y")
df['payment_date'] = pd.to_datetime(df['payment_date'], format="%d/%m/%Y")


df['value'] = df['value'].replace('\.', '',regex=True)
df['value'] = df['value'].replace(',', '.', regex=True)
df.astype({'value': 'float'})

df['city'] = 'tiradentes'
df.head()

TypeError: to_datetime() got an unexpected keyword argument 'inplace'

In [33]:
import pandas as pd
from sqlalchemy import create_engine, URL, text

url = URL.create(
    "postgresql+psycopg2",
    username='postgres',
    password='postgres',
    host='localhost',
    port=5432,
    database='gov_transparency_hub',
)

conn = create_engine(url).connect()

df.to_sql('expense', conn, if_exists='append', index=False)

249